# Conditional Gaussian on a sphere

## Setup

In [1]:
%matplotlib inline

import sys
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


## Load results

In [2]:
setup_filenames = [
    "2_3_0.010",
    "2_3_0.001",
    "2_3_0.100",
]
setup_labels = [
    "0.01",
    "0.001",
    "0.1",
]
n_runs = 3

In [3]:
algo_filenames = []
algo_additionals = []
algo_labels = []
algo_dividers = []

def add_algo(filename, add, label):
    algo_filenames.append(filename)
    algo_additionals.append(add)
    algo_labels.append(label)
    
    
def add_divider():
    algo_dividers.append(len(algo_filenames))
    
    
add_divider()
add_algo("flow", "_small", "Flow")
add_algo("flow", "_small_long", "Flow (long)")
add_algo("flow", "_small_shallow_long", "Flow (shallow, long)")

add_divider()
add_algo("mf_specified", "_small", "MF") 
add_algo("mf_specified", "_small_long", "MF (long)") 
add_algo("mf_specified", "_small_shallow_long", "MF (shallow, long)")

add_divider()
add_algo("gamf_specified", "_small_largebs", "MF-A")
add_algo("gamf_specified", "_small_hugebs", "MF-A (5k batchsize)") 
add_algo("gamf_specified", "_small_largebs_long", "MF-A (long)") 
add_algo("gamf_specified", "_small_largebs_shallow_long", "MF-A (shallow, long)") 

# add_algo("pie_specified", "_small", "Prescr. PIE") 
# add_algo("pie_specified", "_small_long", "Prescr. PIE (long)") 
# add_algo("pie_specified", "_small_shallow_long", "Prescr. PIE (shallow, long)") 

add_divider()
add_algo("pie", "_small", "PIE") 
# add_algo("pie", "_small_pieepsilon03", "PIE (ε = 0.3)") 
# add_algo("pie", "_small_pieepsilon01", "PIE (ε = 0.1)") 
# add_algo("pie", "_small_pieepsilon003", "PIE (ε = 0.03)") 
# add_algo("pie", "_small_pieepsilon0003", "PIE (ε = 0.003)") 
# add_algo("pie", "_small_pieepsilon0001", "PIE (ε = 0.001)") 
add_algo("pie", "_small_long", "PIE (long)") 
add_algo("pie", "_small_shallow_long", "PIE (shallow, long)") 

add_divider()
add_algo("mf", "_small", "MLF")
add_algo("mf", "_small_noprepost", "MLF (no pre/post)")
add_algo("mf", "_small_complex", "MLF (complex)") 
add_algo("mf", "_small_prepie", "MLF (pre-PIE)")
add_algo("mf", "_small_prepie_long", "MLF (pre-PIE, long)")
add_algo("mf", "_small_morenll", "MLF (NLL-heavy)")
add_algo("mf", "_small_morenll_long", "MLF (NLL-heavy, long)")
add_algo("mf", "_small_long", "MLF (long)")
add_algo("mf", "_small_shallow_long", "MLF (shallow, long)")

add_divider()
add_algo("emf", "_small", "MLF-E")
add_algo("emf", "_small_long", "MLF-E (long)")
add_algo("emf", "_small_shallow_long", "MLF-E (shallow, long)")

add_divider()
add_algo("gamf", "_small_largebs", "MLF-A")
add_algo("gamf", "_small_hugebs", "MLF-A (5k batchsize)") 
add_algo("gamf", "_small_largebs_long", "MLF-A (long)") 
add_algo("gamf", "_small_largebs_shallow_long", "MLF-A (shallow, long)") 


In [4]:
def load(quantity, shape, numpyfy=True, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add in zip(algo_filenames, algo_additionals):
        results = []
        
        for setup_filename in setup_filenames:
            try:
                results.append(np.load(
                    "{}/{}_2_conditional_spherical_gaussian_{}{}_{}.npy".format(
                        result_dir, algo_filename, setup_filename, algo_add, quantity
                    )
                ))
            except FileNotFoundError as e:
                print(e)
                results.append(np.nan*np.ones(shape))
            
        all_results.append(results)
    
    return np.asarray(all_results) if numpyfy else all_results

model_posterior_samples = load("model_posterior_samples", (1000, 2))
model_mmds = load("mmd", (1,))

model_test_log_likelihood = load("model_log_likelihood_test", (11*11, 1000,))
model_test_reco_error = load("model_reco_error_test", (1000,))
parameter_grid = load("parameter_grid_test", (11*11,2))

model_test_log_likelihood.shape

[Errno 2] No such file or directory: '../data/results/gamf_specified_2_conditional_spherical_gaussian_2_3_0.010_small_hugebs_model_posterior_samples.npy'
[Errno 2] No such file or directory: '../data/results/gamf_specified_2_conditional_spherical_gaussian_2_3_0.001_small_hugebs_model_posterior_samples.npy'
[Errno 2] No such file or directory: '../data/results/gamf_specified_2_conditional_spherical_gaussian_2_3_0.100_small_hugebs_model_posterior_samples.npy'
[Errno 2] No such file or directory: '../data/results/gamf_specified_2_conditional_spherical_gaussian_2_3_0.010_small_largebs_long_model_posterior_samples.npy'
[Errno 2] No such file or directory: '../data/results/gamf_specified_2_conditional_spherical_gaussian_2_3_0.001_small_largebs_long_model_posterior_samples.npy'
[Errno 2] No such file or directory: '../data/results/gamf_specified_2_conditional_spherical_gaussian_2_3_0.100_small_largebs_long_model_posterior_samples.npy'
[Errno 2] No such file or directory: '../data/results/gamf

(29, 3, 121, 1000)

In [ ]:
def load_truth(name, samples=True):
    if samples:
        return np.asarray([
            [
                np.load("../data/samples/conditional_spherical_gaussian/spherical_gaussian_{}_{}{}.npy".format(
                    setup_filename, name, run_str
                ))
                for run_str in [""] + ["_run{}".format(i) for i in range(1, n_runs)]
            ]
            for setup_filename in setup_filenames
        ])
    else:
        return np.asarray([
            [
                np.load("../data/results/truth_conditional_spherical_gaussian_{}{}_{}.npy".format(
                    setup_filename, run_str, name
                ))
                for run_str in [""] + ["_run{}".format(i) for i in range(1, n_runs)]
            ]
            for setup_filename in setup_filenames
        ])
    
    
test_x = load_truth("x_test", True)
test_distances = np.abs(np.sum(x_test**2, axis=-1)**0.5 - 1.)
test_log_likelihood = load_truth("true_log_likelihood_test", False)
true_posterior_samples = load_truth("true_posterior_samples", False)

## Calculate metrics

In [6]:
true_expected_nll = -2. * np.mean(true_test_log_likelihood, axis=-1)
model_expected_nll = -2. * np.mean(model_test_log_likelihood, axis=-1)

n_observed = 20
true_observed_nll = -2. * np.sum(true_test_log_likelihood[:,:,:,:n_observed], axis=-1)
model_observed_nll = -2. * np.sum(model_test_log_likelihood[:,:,:,:n_observed], axis=-1)

## Print metrics

In [7]:
def print_results(setup):
    print("{:<36.36s} | {:>8.8s}".format(setup_labels[setup], "MMD"))
    print("-"*47)
    for label, mmd in zip(algo_labels, mmds):
        if np.isfinite(logp[setup]) or np.isfinite(dist[setup]):
            print("{:<36.36s} | {:>8.2f}".format(label, mmd[setup]))
        else:
            print("{:<36.36s} |         ".format(label))

In [8]:
def print_all_results():
    print(
        "{:<36.36s} | {:>8.8s} | {:>8.8s} | {:>8.8s}".format(
        "epsilon", setup_labels[0], setup_labels[1], setup_labels[2]
        )
    )
    print(
        "{:<36.36s} | {:>8.8s} | {:>8.8s} | {:>8.8s}".format(
        "", "MMD", "MMD", "MMD"
        )
    )
    print("-"*69)
    for label, mmd in zip(
        algo_labels,
        mmds
    ):
        def _f(val):
            return "{:>8.3f}".format(val) if np.isfinite(val) else "        "
        
        print(
            "{:<36.36s} | {} | {} | {}".format(
                label, _f(mmd[0]), _f(mmd[1]), _f(mmd[2])
            )
        )
        

In [9]:
print_all_results()

epsilon                              |     0.01 |    0.001 |      0.1
                                     |      MMD |      MMD |      MMD
---------------------------------------------------------------------
Flow                                 |    0.391 |    0.285 |    0.099
Flow (long)                          |    1.262 |    1.070 |    0.359
Flow (shallow, long)                 |    0.456 |    0.220 |    0.038
PIE                                  |    0.321 |    0.008 |    0.149
PIE (long)                           |    0.934 |    0.959 |    0.258
PIE (shallow, long)                  |    1.066 |    0.861 |    0.091
MAD AF                               |    0.367 |    0.009 |    0.012
MAD AF (no pre/post)                 |    0.796 |    1.218 |    0.216
MAD AF (complex)                     |    1.354 |    0.689 |    0.646
MAD AF (long)                        |          |          |         
MAD AF (shallow, long)               |    1.330 |          |         
MAD AF + Enc.       

In [ ]:
print_results(1)

In [ ]:
print_results(2)